In [1]:
import random
import warnings
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from tqdm.auto import tqdm
from scipy.optimize import minimize

In [2]:
%matplotlib notebook
plt.rcParams["figure.figsize"] = (15, 12)

In [3]:
warnings.filterwarnings("ignore")

In [4]:
def rand(a, b):
    return a + (b - a) * random.random()

In [5]:
ALPHA = 0.01

In [6]:
basis = np.array([
    [1., 0., 0.], 
    [0., 1., 0.], 
    [0., 0., 1.]
])

In [7]:
db_neg = np.array([
    [1. - ALPHA, 0., 0.], 
    [0., 1. - ALPHA, 0.], 
    [0., 0., 1. - ALPHA]
])
db_pos = np.array([
    [1. + ALPHA, 0., 0.], 
    [0., 1. + ALPHA, 0.], 
    [0., 0., 1. + ALPHA]
])

In [8]:
dc11_neg = np.array([
    [1. - ALPHA, 0., 0.], 
    [0., 1., 0.], 
    [0., 0., 1.]
])
dc11_pos = np.array([
    [1. + ALPHA, 0., 0.], 
    [0., 1., 0.], 
    [0., 0., 1.]
])

In [9]:
dc22_neg = np.array([
    [1., 0., 0.], 
    [0., 1. - ALPHA, 0.], 
    [0., 0., 1.]
])
dc22_pos = np.array([
    [1., 0., 0.], 
    [0., 1. + ALPHA, 0.], 
    [0., 0., 1.]
])

In [10]:
dc44_neg = np.array([
    [1. / (1. - ALPHA * ALPHA) ** (1/3), 0., 0.], 
    [0., 1. / (1. - ALPHA * ALPHA) ** (1/3), ALPHA / (1 - ALPHA * ALPHA) ** (1/3)], 
    [0., ALPHA / (1. - ALPHA * ALPHA) ** (1/3), 1. / (1 - ALPHA * ALPHA) ** (1/3)]
])
dc44_pos = np.array([
    [1. / (1.- ALPHA * ALPHA) ** (1/3), 0., 0.], 
    [0., 1. / (1. - ALPHA * ALPHA) ** (1/3), -ALPHA / (1. - ALPHA * ALPHA) ** (1/3)], 
    [0., -ALPHA / (1. - ALPHA * ALPHA) ** (1/3), 1. / (1. - ALPHA * ALPHA) ** (1/3)]
])

In [11]:
dc12_neg = np.array([
    [(1. + ALPHA) / (1 - ALPHA * ALPHA) ** (1/3), 0., 0.], 
    [0., (1. - ALPHA) / (1 - ALPHA * ALPHA) ** (1/3), 0.], 
    [0., 0., 1. / (1 - ALPHA * ALPHA) ** (1/3)]
])
dc12_pos = np.array([
    [(1. - ALPHA) / (1. - ALPHA * ALPHA) ** (1/3), 0., 0.], 
    [0., (1. + ALPHA) / (1. - ALPHA * ALPHA) ** (1/3), 0.], 
    [0., 0., 1. / (1. - ALPHA * ALPHA) ** (1/3)]
])

# RGL

In [12]:
unit_cell = np.array([
    [0.00, 0.00, 0.00],
    [0.00, 0.50, 0.50],
    [0.50, 0.00, 0.50],
    [0.50, 0.50, 0.00],
])

In [13]:
rep = 3
super_cell = np.vstack([
    unit_cell + np.array([[i, j, k]])
    for i in range(rep) for j in range(rep) for k in range(rep)]) / rep

Если увеличить rep до 4, то в Test RGL 1 и 2 получаются точные результаты

In [14]:
min_dist = np.inf
for i in range(super_cell.shape[0]):
    for j in range(i, super_cell.shape[0]):
        dist = np.sqrt((super_cell[i][0] - super_cell[j][0]) ** 2 + (super_cell[i][1] - super_cell[j][1]) ** 2 + (super_cell[i][2] - super_cell[j][2]) ** 2)
        if dist != 0 and dist < min_dist:
            min_dist = dist

In [15]:
def f(r, R=6.5, D=0.5):
    if r < R - D:
        return 1
    elif R - D <= r and r < R + D:
        return 1 / 2 * (1 - np.sin(np.pi / 2 * (r - R) / D))
    elif r >= R + D:
        return 0

In [16]:
def rgl_potential(matrix, lattice_constant, alpha, ksi, p, q, r0, R=5.0, D=0.5):
    full_energy = 0.
    for i in range(unit_cell.shape[0]):
        energy_r = 0.
        energy_b = 0.
        for j in range(super_cell.shape[0]):
            if i == j:
                continue
                
            direction = matrix @ (np.remainder(super_cell[i]- super_cell[j] + 0.5, 1) - 0.5)
            distance = np.linalg.norm(direction) * lattice_constant
            
            if distance < (R + D):
                energy_r += f(distance) * alpha * np.exp(-1 * p * (distance / r0 - 1))
                energy_b += f(distance) * ksi * ksi * np.exp(-2 * q * (distance / r0 - 1))

        energy_b = -1 * np.sqrt(energy_b)
        full_energy += energy_r + energy_b  
    
    return full_energy

## Test RGL 1

In [17]:
d = np.array([
    [1., 0., 0.],
    [0., 1., 0.],
    [0., 0., 1.]
])

In [18]:
lattice_constant = 4.085
alpha = 0.1028
ksi = 1.178
p = 10.928
q = 3.139
r0 = 2.89

In [19]:
rgl_potential(d, lattice_constant * rep, alpha, ksi, p, q, r0) / unit_cell.shape[0]

-2.9548722415511564

In [20]:
# -2.9349427932749057 при rep 4 можно получить точное значение

## Test RGL 2

In [21]:
d = np.array([
    [1., 0.05, 0.],
    [0.05, 1., 0.],
    [0., 0., 1.]
])

In [22]:
lattice_constant = 4.085
alpha = 0.1028
ksi = 1.178
p = 10.928
q = 3.139
r0 = 2.89

In [23]:
rgl_potential(d, lattice_constant * rep, alpha, ksi, p, q, r0) / unit_cell.shape[0]

-2.927252592396284

In [24]:
# -2.908037481350278 при rep 4 можно получить точное значение

# Test - Ag 

Запустил 3 лабораторную для серебра для получения B, C{ij} чтобы мог точно оценить работает ли мой код, вроде все ок

In [25]:
lattice_constant = 4.085
alpha = 0.1028
ksi = 1.178
p = 10.928
q = 3.139
r0 = 2.89

volume = lattice_constant * lattice_constant * lattice_constant  
energy = rgl_potential(basis, lattice_constant * rep, alpha, ksi, p, q, r0)

In [26]:
print(energy / unit_cell.shape[0])
# E_coh = -2.9603006739221542

-2.9548722415511564


In [27]:
energy_bneg = rgl_potential(db_neg, lattice_constant * rep, alpha, ksi, p, q, r0)
energy_bpos = rgl_potential(db_pos, lattice_constant * rep, alpha, ksi, p, q, r0)
print(160.2 * (energy_bneg - 2 * energy + energy_bpos) / (ALPHA ** 2 * 9 * volume) )
# B = 112.95044925734544

109.66061926736434


In [28]:
energy_c11neg = rgl_potential(dc11_neg, lattice_constant * rep, alpha, ksi, p, q, r0)
energy_c11pos = rgl_potential(dc11_pos, lattice_constant * rep, alpha, ksi, p, q, r0)
print(160.2 * (energy_c11neg - 2 * energy + energy_c11pos) / (ALPHA ** 2  * volume)) 
# C11 = 134.02536227402854

133.80286002675572


In [29]:
energy_c22neg = rgl_potential(dc22_neg, lattice_constant * rep, alpha, ksi, p, q, r0)
energy_c22pos = rgl_potential(dc22_pos, lattice_constant * rep, alpha, ksi, p, q, r0) 
print(160.2 * (energy_c22neg - 2 * energy + energy_c22pos) / (ALPHA ** 2  * volume))
# C22 = 134.02536227402854

133.80286002679748


In [30]:
energy_c44neg = rgl_potential(dc44_neg, lattice_constant * rep, alpha, ksi, p, q, r0)
energy_c44pos = rgl_potential(dc44_pos, lattice_constant * rep, alpha, ksi, p, q, r0) 
print(160.2 * (energy_c44neg - 2 * energy + energy_c44pos) / (ALPHA ** 2  * volume * 4))
# C44 = 51.80549003213346

51.40417869083868


In [31]:
energy_c12neg = rgl_potential(dc12_neg, lattice_constant * rep, alpha, ksi, p, q, r0)
energy_c12pos = rgl_potential(dc12_pos, lattice_constant * rep, alpha, ksi, p, q, r0)
print(160.2 * (energy_c11neg + energy_c11pos + energy_c22neg + energy_c22pos - 2 * energy - energy_c12neg - energy_c12pos) / (ALPHA ** 2  * volume * 2))
# C12 = 101.76283945318289

97.73560790277456


# Params - Ca

In [32]:
lattice_constant = 5.52359108808
E_coh = -1.9089984885038827
b = 17.440890573844225
c11 = 21.603097766183534
c22 = 21.603097766183534
c44 = 14.082393170032377
c12 = 15.34943323587179

In [33]:
properties = {
    'rep': rep,
    'lattice_constant': lattice_constant,
    'E_coh': E_coh,
    'b': b,
    'c11': c11,
    'c22': c22,
    'c44': c44,
    'c12': c12
}

# Loss

In [34]:
def loss(params, properties):
    alpha = params[0]
    ksi = params[1]
    p = params[2]
    q = params[3]
    r0 = params[4]
    
    lattice_constant = properties['lattice_constant']
    rep =  properties['rep']
    
    E_coh = properties['E_coh']
    B = properties['b']
    C11 = properties['c11']
    C22 = properties['c22']
    C44 = properties['c44']
    C12 = properties['c12']
    
    
    volume = lattice_constant * lattice_constant * lattice_constant
    
    energy = rgl_potential(basis, lattice_constant * rep, alpha, ksi, p, q, r0)
    
    energy_bneg = rgl_potential(db_neg, lattice_constant * rep, alpha, ksi, p, q, r0)
    energy_bpos = rgl_potential(db_pos, lattice_constant * rep, alpha, ksi, p, q, r0)
    
    energy_c11neg = rgl_potential(dc11_neg, lattice_constant * rep, alpha, ksi, p, q, r0)
    energy_c11pos = rgl_potential(dc11_pos, lattice_constant * rep, alpha, ksi, p, q, r0)
    
    energy_c22neg = rgl_potential(dc22_neg, lattice_constant * rep, alpha, ksi, p, q, r0)
    energy_c22pos = rgl_potential(dc22_pos, lattice_constant * rep, alpha, ksi, p, q, r0)
    
    energy_c44neg = rgl_potential(dc44_neg, lattice_constant * rep, alpha, ksi, p, q, r0)
    energy_c44pos = rgl_potential(dc44_pos, lattice_constant * rep, alpha, ksi, p, q, r0)
    
    energy_c12neg = rgl_potential(dc12_neg, lattice_constant * rep, alpha, ksi, p, q, r0)
    energy_c12pos = rgl_potential(dc12_pos, lattice_constant * rep, alpha, ksi, p, q, r0)
    
    E_coh_cur = energy / unit_cell.shape[0]
    
    B_cur = 160.2 * (energy_bneg - 2 * energy + energy_bpos) / (ALPHA ** 2 * 9 * volume) 
    C11_cur = 160.2 * (energy_c11neg - 2 * energy + energy_c11pos) / (ALPHA ** 2  * volume)
    C22_cur = 160.2 * (energy_c22neg - 2 * energy + energy_c22pos) / (ALPHA ** 2  * volume)
    C44_cur = 160.2 * (energy_c44neg - 2 * energy + energy_c44pos) / (ALPHA ** 2  * volume * 4)
    C12_cur = 160.2 * (energy_c11neg + energy_c11pos + energy_c22neg + energy_c22pos - 2 * energy - energy_c12neg - energy_c12pos) / (ALPHA ** 2  * volume * 2)

    
    error = ((E_coh - E_coh_cur) / abs(E_coh)) ** 2 + \
            ((B - B_cur) / abs(B)) ** 2 + \
            ((C11 - C11_cur) / abs(C11)) ** 2 + \
            ((C22 - C22_cur) / abs(C22)) ** 2 + \
            ((C44 - C44_cur) / abs(C44)) ** 2 + \
            ((C12 - C12_cur) / abs(C12)) ** 2
    
    return error

# Optimization

In [36]:
md = min_dist * properties['lattice_constant'] * rep

## Nelder - Mead

In [35]:
# With Bounds

In [37]:
bnds = [
    (0.01, 1.), (0.5, 5.0), (5., 40.), (0.5, 10.), (0.95 * md, 1.05 * md)
]

column_names = ["alpha0", "ksi0", "p0", "q0", "r0", "alpha", "ksi", "p", "q", "r", "error"]
df = pd.DataFrame(columns = column_names)

for i in tqdm(range(200)):
    
    rand_alpha = rand(0.01, 1.)
    rand_ksi = rand(0.5, 5.0)
    rand_p = rand(5., 40.)
    rand_q = rand(0.5, 10.)
    rand_r0 = rand(0.95, 1.05) * md
    
    params = [rand_alpha, rand_ksi, rand_p, rand_q, rand_r0]

    result = minimize(
        loss, 
        params, 
        args= (properties, ), 
        method='Nelder-Mead', 
        bounds=bnds,
        options={
            'maxiter':10000, 
        }
    )
    
    df.loc[i] = [rand_alpha, rand_ksi, rand_p, rand_q, rand_r0, result.x[0], result.x[1], result.x[2], result.x[3], result.x[4], result.fun]
    
df.to_csv('Nelder_Mead_With.csv', index=False)
display(df)

  0%|          | 0/200 [00:00<?, ?it/s]

,alpha0,ksi0,p0,q0,r0,alpha,ksi,p,q,r,error
0,0.843657,1.895298,22.737781,9.039422,3.745253,0.048164,0.500000,40.000000,6.359409,3.710480,0.545233
1,0.943960,4.218664,22.781488,6.583295,3.861186,0.136187,0.716487,6.082825,2.912193,3.710480,1.095628
2,0.789284,2.772991,13.948422,1.066810,3.755514,0.116695,0.665419,6.237233,2.986120,3.804700,1.095628
3,0.282344,4.078869,27.668859,8.149006,3.853368,0.048163,0.500000,40.000000,6.359348,3.710480,0.545233
4,0.814786,3.190598,22.958381,3.551724,4.075438,0.128104,0.695803,6.143962,2.941458,3.747801,1.095628
...,...,...,...,...,...,...,...,...,...,...,...
195,0.538094,4.248323,15.328017,5.778207,3.924283,0.048160,0.500000,39.999983,6.358910,3.710480,0.545233
196,0.472707,4.609081,28.085294,4.783683,4.060362,0.352641,5.000000,39.999993,6.674777,3.710480,73.500623
197,0.094634,4.847780,10.474073,4.164334,3.966328,0.110178,0.500000,7.492317,3.493998,3.710480,1.129631
198,0.184242,1.891599,30.595642,8.054998,3.839429,0.048163,0.500000,39.999995,6.359377,3.710480,0.545233


In [38]:
# Without bounds

In [39]:
column_names = ["alpha0", "ksi0", "p0", "q0", "r0", "alpha", "ksi", "p", "q", "r", "error"]
df = pd.DataFrame(columns = column_names)

for i in tqdm(range(200)):
    
    rand_alpha = rand(0.01, 1.)
    rand_ksi = rand(0.5, 5.0)
    rand_p = rand(5., 40.)
    rand_q = rand(0.5, 10.)
    rand_r0 = rand(0.95, 1.05) * md
    
    params = [rand_alpha, rand_ksi, rand_p, rand_q, rand_r0]

    result = minimize(
        loss, 
        params, 
        args= (properties, ), 
        method='Nelder-Mead', 
        options={
            'maxiter':10000, 
        }
    )
    
    df.loc[i] = [rand_alpha, rand_ksi, rand_p, rand_q, rand_r0, result.x[0], result.x[1], result.x[2], result.x[3], result.x[4], result.fun]
    
df.to_csv('Nelder_Mead_Without.csv', index=False)
display(df)

  0%|          | 0/200 [00:00<?, ?it/s]

,alpha0,ksi0,p0,q0,r0,alpha,ksi,p,q,r,error
0,0.281129,3.295559,5.942426,1.395588,3.796860,0.367855,3.816469,6.235227,2.235732,0.062137,6.000000
1,0.487485,2.660676,11.045162,4.003630,4.068272,0.880362,0.533831,70.863407,6.140635,3.584875,0.415115
2,0.257274,3.541771,22.621402,4.908748,3.949880,1.855188,0.569450,70.117927,6.076042,3.547166,0.415115
3,0.715109,4.908863,36.821984,9.461593,3.916602,8.754248,0.651398,68.566411,5.941593,3.468675,0.415115
4,0.387160,0.604002,32.341404,9.042016,3.797946,0.003274,0.323275,70.460299,6.617660,3.873683,0.416902
...,...,...,...,...,...,...,...,...,...,...,...
195,0.847226,4.495896,36.214406,8.295954,3.987953,0.590901,0.515703,71.262094,6.175183,3.605044,0.415115
196,0.863533,3.128277,10.345874,1.418554,3.830057,0.921979,1.426551,13.518997,2.337350,0.142854,6.000000
197,0.023133,2.833463,14.527318,2.672115,3.767405,0.022996,3.344638,18.839737,3.968728,0.011406,6.000000
198,0.582148,4.748462,32.524972,6.792800,4.096660,0.755683,0.526813,71.016168,6.153869,3.592601,0.415115


## Powell

In [40]:
# With Bounds

In [42]:
bnds = [
    (0.01, 1.), (0.5, 5.0), (5., 40.), (0.5, 10.), 
    (0.95 * md, 1.05 * md)
]

column_names = ["alpha0", "ksi0", "p0", "q0", "r0", "alpha", "ksi", "p", "q", "r", "error"]
df = pd.DataFrame(columns = column_names)

for i in tqdm(range(50)):
    
    rand_alpha = rand(0.01, 1.)
    rand_ksi = rand(0.5, 5.0)
    rand_p = rand(5., 40.)
    rand_q = rand(0.5, 10.)
    rand_r0 = rand(0.95, 1.05) * md
    
    params = [rand_alpha, rand_ksi, rand_p, rand_q, rand_r0]

    result = minimize(
        loss, 
        params, 
        args= (properties, ), 
        method='powell', 
        bounds=bnds,
        options={
            'maxiter':10000, 
        }
    )
    
    df.loc[i] = [rand_alpha, rand_ksi, rand_p, rand_q, rand_r0, result.x[0], result.x[1], result.x[2], result.x[3], result.x[4], result.fun]
    
df.to_csv('Powell_With.csv', index=False)
display(df)

  0%|          | 0/50 [00:00<?, ?it/s]

,alpha0,ksi0,p0,q0,r0,alpha,ksi,p,q,r,error
0,0.071102,1.379202,21.764233,8.429855,3.791105,0.081514,0.790433,26.870256,7.205553,3.714044,2.393119
1,0.960332,2.047489,6.980550,4.373378,3.720533,0.182999,2.024246,39.999958,7.987263,3.710480,19.169174
2,0.233115,3.349489,31.378054,0.712204,3.832258,0.990885,0.697350,11.207491,1.885056,3.849916,2222.226927
3,0.310129,3.473309,37.769087,8.631234,3.806278,0.059665,0.500078,23.488713,6.372891,3.710480,0.944107
4,0.622357,3.265397,10.964985,5.572182,3.849140,0.367867,2.627798,12.667984,6.507380,3.710480,30.977958
5,0.025558,1.028538,38.181025,6.664073,4.038530,0.010035,0.500000,27.016652,6.957578,4.030239,2.019355
6,0.028084,4.890546,27.514725,7.165344,4.081021,0.080891,0.670422,21.034815,6.452311,3.710512,1.580953
7,0.954302,2.461953,14.225692,4.009582,3.923203,0.144267,1.305145,16.273593,9.984190,4.049478,136.613244
8,0.441761,4.765821,6.378165,4.274844,3.908244,0.077352,0.500063,17.422043,7.439988,3.738487,2.214014
9,0.329850,2.959500,24.159030,7.303334,4.085402,0.033720,0.500177,40.000000,6.472039,3.749318,0.587427


In [44]:
# Without bounds

In [43]:
column_names = ["alpha0", "ksi0", "p0", "q0", "r0", "alpha", "ksi", "p", "q", "r", "error"]
df = pd.DataFrame(columns = column_names)

for i in tqdm(range(50)):
    
    rand_alpha = rand(0.01, 1.)
    rand_ksi = rand(0.5, 5.0)
    rand_p = rand(5., 40.)
    rand_q = rand(0.5, 10.)
    rand_r0 = rand(0.95, 1.05) * md
    
    params = [rand_alpha, rand_ksi, rand_p, rand_q, rand_r0]

    result = minimize(
        loss, 
        params, 
        args= (properties, ), 
        method='powell', 
        options={
            'maxiter':10000, 
        }
    )
    
    df.loc[i] = [rand_alpha, rand_ksi, rand_p, rand_q, rand_r0, result.x[0], result.x[1], result.x[2], result.x[3], result.x[4], result.fun]
    
df.to_csv('Powell_Without.csv', index=False)
display(df)

  0%|          | 0/50 [00:00<?, ?it/s]

,alpha0,ksi0,p0,q0,r0,alpha,ksi,p,q,r,error
0,0.799916,2.883512,14.987768,6.081855,4.093317,0.398789,0.990215,6.197918,2.927875,3.096984,1.119477e+00
1,0.255042,1.966364,18.369522,5.140240,3.964311,0.148972,0.453017,71.353461,6.281039,3.672088,4.153259e-01
2,0.850258,1.353828,35.542422,3.820948,3.789644,2.708019,3.551949,29.334092,7.026911,-0.427208,inf
3,0.288968,1.606761,5.578691,9.270284,4.020006,0.050487,0.970854,7.196725,5.131374,-0.142485,9.781038e+250
4,0.957075,4.866738,6.735244,1.163111,3.741652,0.031529,0.245167,-36.570016,2.092069,5.536789,1.990221e+00
5,0.549753,0.845566,16.456675,6.277370,3.841988,1.148948,0.628809,58.073093,6.051897,3.529612,4.273684e-01
6,0.059663,0.931678,36.829572,9.161473,3.827227,0.033338,0.401968,74.128966,6.424295,3.750476,4.151149e-01
7,0.276739,2.285274,34.399494,5.122953,3.974147,0.001918,-0.181281,65.361514,5.911946,3.893503,5.895462e-01
8,0.101464,3.461151,32.062540,1.152073,4.040712,5.629851,7.386681,32.850044,6.945965,-0.376070,inf
9,0.346929,3.265327,16.688060,1.006131,3.894116,0.924239,0.536126,70.710057,6.135315,3.582095,4.151156e-01


## SLSQP

In [45]:
# With Bounds

In [46]:
bnds = [
    (0.01, 1.), (0.5, 5.0), (5., 40.), (0.5, 10.), 
    (0.95 * md, 1.05 * md)
]

column_names = ["alpha0", "ksi0", "p0", "q0", "r0", "alpha", "ksi", "p", "q", "r", "error"]
df = pd.DataFrame(columns = column_names)

for i in tqdm(range(50)):
    
    rand_alpha = rand(0.01, 1.)
    rand_ksi = rand(0.5, 5.0)
    rand_p = rand(5., 40.)
    rand_q = rand(0.5, 10.)
    rand_r0 = rand(0.95, 1.05) * md
    
    params = [rand_alpha, rand_ksi, rand_p, rand_q, rand_r0]

    result = minimize(
        loss, 
        params, 
        args= (properties, ), 
        method='SLSQP', 
        bounds=bnds,
        options={
            'maxiter':10000, 
        }
    )
    
    df.loc[i] = [rand_alpha, rand_ksi, rand_p, rand_q, rand_r0, result.x[0], result.x[1], result.x[2], result.x[3], result.x[4], result.fun]
    
df.to_csv('SLSQP.csv', index=False)
display(df)

  0%|          | 0/50 [00:00<?, ?it/s]

,alpha0,ksi0,p0,q0,r0,alpha,ksi,p,q,r,error
0,0.395689,4.833831,32.146312,4.910088,3.715125,0.048160,0.500000,40.000000,6.359586,3.710480,5.452329e-01
1,0.128003,3.068357,7.880832,3.527770,3.904844,0.110483,0.648256,6.291467,3.012123,3.838088,1.095628e+00
2,0.018314,4.578961,7.392060,2.882662,3.889430,0.090274,0.500000,14.518491,6.136083,3.710480,1.573824e+00
3,0.155711,1.955937,29.383337,2.978650,3.999841,0.048163,0.500000,40.000000,6.359289,3.710480,5.452326e-01
4,0.130863,4.742672,34.267050,5.623216,4.092600,0.048163,0.500000,40.000000,6.358842,3.710480,5.452328e-01
5,0.028871,0.812074,19.256662,4.230833,4.097938,0.125542,0.688879,6.166653,2.952134,3.760042,1.095628e+00
6,0.182319,4.152769,27.649478,2.141847,3.929781,0.418457,3.097886,31.216184,4.411203,3.866449,2.106312e+04
7,0.256511,2.971288,25.336740,4.967852,3.888131,0.048164,0.500000,40.000000,6.358902,3.710480,5.452328e-01
8,0.041538,3.332832,7.535546,0.650715,3.804343,0.041538,3.332832,7.535546,0.650715,3.804343,1.926541e+06
9,0.305599,4.983672,32.084742,6.609034,4.047781,0.305599,4.983672,32.084742,6.609034,4.047781,2.783629e+05


# Compare

In [48]:
nelder_mead = pd.read_csv('Nelder_Mead_With.csv')
nelder_mead_without = pd.read_csv('Nelder_Mead_Without.csv')
powell = pd.read_csv('Powell_With.csv')
powell_without = pd.read_csv('Powell_Without.csv')
slsqp = pd.read_csv('SLSQP.csv')

In [51]:
nelder_mead.nsmallest(5, 'error')

,alpha0,ksi0,p0,q0,r0,alpha,ksi,p,q,r,error
51,0.469600,1.646828,32.228389,3.779039,3.902663,0.048163,0.5,40.0,6.359383,3.71048,0.545233
40,0.160865,4.673631,31.216199,2.069217,3.913341,0.048163,0.5,40.0,6.359399,3.71048,0.545233
19,0.643282,4.361319,29.744959,7.847369,4.081857,0.048163,0.5,40.0,6.359348,3.71048,0.545233
100,0.037065,2.822614,36.153774,3.866102,3.778499,0.048163,0.5,40.0,6.359348,3.71048,0.545233
161,0.630758,0.842868,29.960733,7.968074,4.002807,0.048163,0.5,40.0,6.359347,3.71048,0.545233


In [52]:
nelder_mead_without.nsmallest(5, 'error')

,alpha0,ksi0,p0,q0,r0,alpha,ksi,p,q,r,error
125,0.069609,3.158172,36.134329,9.205172,3.904584,0.158669,-0.460170,72.576890,6.289119,3.671559,0.415115
186,0.080620,2.204200,37.034392,9.446619,4.049280,0.059329,0.422568,73.560676,6.374364,3.721324,0.415115
100,0.767915,4.496169,9.429843,4.342637,3.898828,5.163789,0.622273,69.094319,5.987335,3.495379,0.415115
1,0.487485,2.660676,11.045162,4.003630,4.068272,0.880362,0.533831,70.863407,6.140635,3.584875,0.415115
89,0.659027,4.734088,22.300343,5.923001,4.041427,0.039271,-0.407726,73.973242,6.410118,3.742198,0.415115


In [53]:
powell.nsmallest(5, 'error')

,alpha0,ksi0,p0,q0,r0,alpha,ksi,p,q,r,error
28,0.924052,3.014850,32.027914,9.117198,4.056344,0.041782,0.500012,39.999899,6.391227,3.725323,0.557320
32,0.872727,4.374407,39.965225,6.160235,3.923232,0.049158,0.500370,40.000000,6.399640,3.710480,0.565086
10,0.919606,3.496730,37.495998,6.103548,3.903723,0.048252,0.500000,36.855927,6.361273,3.710496,0.581616
9,0.329850,2.959500,24.159030,7.303334,4.085402,0.033720,0.500177,40.000000,6.472039,3.749318,0.587427
23,0.175127,1.225098,6.494666,9.390053,3.937719,0.023946,0.500037,40.000000,6.516495,3.785024,0.623439


In [54]:
powell_without.nsmallest(5, 'error')

,alpha0,ksi0,p0,q0,r0,alpha,ksi,p,q,r,error
44,0.820607,2.333039,36.066195,7.826845,3.858027,0.050417,0.416624,73.717827,6.388398,3.729550,0.415115
6,0.059663,0.931678,36.829572,9.161473,3.827227,0.033338,0.401968,74.128966,6.424295,3.750476,0.415115
15,0.100932,4.817695,33.454251,6.119570,4.028489,0.023398,-0.389821,74.505579,6.454953,3.768399,0.415115
9,0.346929,3.265327,16.688060,1.006131,3.894116,0.924239,0.536126,70.710057,6.135315,3.582095,0.415116
37,0.256552,1.920721,32.333787,5.617313,3.765218,0.049006,0.415997,73.578449,6.390861,3.730849,0.415117


In [55]:
slsqp.nsmallest(5, 'error')

,alpha0,ksi0,p0,q0,r0,alpha,ksi,p,q,r,error
48,0.669379,4.914341,38.685410,6.629673,3.723092,0.048163,0.5,40.0,6.359364,3.71048,0.545233
29,0.129659,2.825470,20.690836,9.343868,4.050586,0.048163,0.5,40.0,6.359434,3.71048,0.545233
20,0.518908,2.511361,29.577746,8.700195,3.968931,0.048163,0.5,40.0,6.359441,3.71048,0.545233
15,0.084793,0.810634,28.783285,1.071718,3.965083,0.048164,0.5,40.0,6.359326,3.71048,0.545233
3,0.155711,1.955937,29.383337,2.978650,3.999841,0.048163,0.5,40.0,6.359289,3.71048,0.545233


In [58]:
import dataframe_image as dfi

In [60]:
dfi.export(nelder_mead.nsmallest(5, 'error'), 'nelder_mead.png')
dfi.export(nelder_mead_without.nsmallest(5, 'error'), 'nelder_mead_without.png')
dfi.export(powell.nsmallest(5, 'error'), 'powell.png')
dfi.export(powell_without.nsmallest(5, 'error'), 'powell_without.png')
dfi.export(slsqp.nsmallest(5, 'error'), 'slsqp.png')

objc[60298]: Class WebSwapCGLLayer is implemented in both /System/Library/Frameworks/WebKit.framework/Versions/A/Frameworks/WebCore.framework/Versions/A/Frameworks/libANGLE-shared.dylib (0x7ffa5a372318) and /Applications/Google Chrome.app/Contents/Frameworks/Google Chrome Framework.framework/Versions/101.0.4951.64/Libraries/libGLESv2.dylib (0x112ce1218). One of the two will be used. Which one is undefined.
[0530/024100.600020:INFO:headless_shell.cc(660)] Written to file /var/folders/c9/cy_fzlsd5wxftq5myg_bgvhm0000gn/T/tmpelogdhtz/temp.png.
objc[60311]: Class WebSwapCGLLayer is implemented in both /System/Library/Frameworks/WebKit.framework/Versions/A/Frameworks/WebCore.framework/Versions/A/Frameworks/libANGLE-shared.dylib (0x7ffa5a372318) and /Applications/Google Chrome.app/Contents/Frameworks/Google Chrome Framework.framework/Versions/101.0.4951.64/Libraries/libGLESv2.dylib (0x10e677218). One of the two will be used. Which one is undefined.
[0530/024102.402757:INFO:headless_shell.cc(6